In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as Fb
import pytorch_lightning as pl

from torch.nn import functional as F
from torch.optim import Adam
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms


In [2]:
class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        
        # logs metrics for each training_step,
        # and the average across the epoch, to the progress bar and logger
        return self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

    def configure_optimizers(self):
        return Adam(self.parameters(), lr=0.02)

In [3]:
train_loader = DataLoader(datasets.MNIST(os.getcwd(), download=True,transform=transforms.ToTensor()),num_workers=12)
trainer = pl.Trainer(accelerator='gpu', max_epochs=3)  #gpus=1,fast_dev_run=True
model = LitModel()

trainer.fit(model, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params
--------------------------------
0 | l1   | Linear | 7.9 K 
--------------------------------
7.9 K     Trainable params
0         Non-trainable params
7.9 K     Total params
0.031     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

D:\Users\marka\anaconda3\envs\pytorch_ev\lib\site-packages\pytorch_lightning\loops\optimization\optimizer_loop.py:136: UserWarning: `training_step` returned `None`. If this was on purpose, ignore this warning...
  self.warning_cache.warn("`training_step` returned `None`. If this was on purpose, ignore this warning...")
D:\Users\marka\anaconda3\envs\pytorch_ev\lib\site-packages\pytorch_lightning\trainer\call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
